In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score
from sklearn.neural_network import MLPClassifier

In [2]:
url = "D:/1研究生学习资料/实验数据/实验数据/重做/重做-去重/Re去重/关键基因/RNA39作为未证实/final_train_RNA_data795.csv"
data = pd.read_csv(url)

### Load data set

In [3]:
data

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,y
0,1706.619722,-1.758654,0.204940,-8.581328,9.380000e-18,1.270000e-16,1
1,262.779843,1.104679,0.205058,5.387159,7.160000e-08,3.090000e-07,1
2,2881.943486,1.212687,0.392819,3.087136,2.020949e-03,4.306827e-03,1
3,228.182361,3.784003,0.949153,3.986718,6.700000e-05,1.857950e-04,1
4,6866.074347,2.430582,0.491265,4.947604,7.510000e-07,2.820000e-06,1
...,...,...,...,...,...,...,...
790,40.111376,3.860520,0.527058,7.324653,2.400000e-13,2.050000e-12,0
791,2528.928177,2.284136,0.247865,9.215224,3.110000e-20,5.160000e-19,0
792,71.091778,1.006783,0.253426,3.972691,7.110000e-05,1.959630e-04,0
793,300.568538,1.755329,0.335342,5.234437,1.650000e-07,6.800000e-07,0


In [4]:
data.iloc[:, -1].value_counts()

0    713
1     82
Name: y, dtype: int64

In [5]:

mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

Using 82/82 as positives and unlabeling the rest
target variable:
 -1    713
 1     82
Name: class_test, dtype: int64


In [6]:
y_data = mod_data.iloc[:,-1]
df_orig_positive  = mod_data.iloc[y_data.values == 1]#正标签所有行数据
df_orig_unlabeled = mod_data.iloc[y_data.values != 1]#未标记标签所有行数据
x_data_pos = df_orig_positive.iloc[:,:-2].values#正标签特征
x_data_unl = df_orig_unlabeled.iloc[:,:-2].values#未标记标签特征

In [7]:
len_pos = x_data_pos.shape[0] # size of positives  82
len_unlabeled = x_data_unl.shape[0] #size of unlabeled 713
learners_num = 128 #learners
bootstrap_sample_size = len_pos # random bootstrap sample size

In [8]:
#create a label set for each learning cycle
#为每个学习周期创建一个标签集
train_labels = np.zeros(shape=(len_pos + bootstrap_sample_size,)) #164
#populate the first part of the set with the positive label, 
#用正面标签填充第一部分，
train_labels[:len_pos] = 1.0
#place holder array for the number of times the datapoint is predicted
#预测数据点的次数的占位符数组(袋外数据)
n_oob = np.zeros(shape=(len_unlabeled,))
#holds the results of the prediction of the data point
#保存数据点的预测结果
f_oob = np.zeros(shape=(len_unlabeled, 2))

In [49]:
%pip install lightgbm
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
for i in range(learners_num):
    # Bootstrap resample
    bootstrap_sample = np.random.choice(np.arange(len_unlabeled), replace=True, size=bootstrap_sample_size)
    # Positive set + bootstrapped unlabeled set
    data_bootstrap = np.concatenate((x_data_pos,x_data_unl[bootstrap_sample, :]), axis=0)
    # Train model
    #model = lgb.LGBMClassifier()
    model = MLPClassifier(random_state=42, verbose=0,hidden_layer_sizes=9)
    model.fit(data_bootstrap, train_labels)
    # Index for the out of the bag (oob) samples
    idx_oob = sorted(set(range(len_unlabeled)) - set(np.unique(bootstrap_sample)))
    # Transductive learning of oob samples
    f_oob[idx_oob] += model.predict_proba(x_data_unl[idx_oob])
    n_oob[idx_oob] += 1
    if(i%10 == 0): print(f'learner {i}/{learners_num} completed')
        
predicted = f_oob[:, 1]/n_oob


learner 0/128 completed
learner 10/128 completed
learner 20/128 completed
learner 30/128 completed
learner 40/128 completed
learner 50/128 completed
learner 60/128 completed
learner 70/128 completed
learner 80/128 completed
learner 90/128 completed
learner 100/128 completed
learner 110/128 completed
learner 120/128 completed


In [52]:
df_orig_predicted = df_orig_unlabeled.copy()
df_orig_predicted['pred'] = [1 if x > 0.9 else 0 for x in predicted]
df_orig_positive.loc[:,'pred'] = 1
df_outcome = pd.concat([df_orig_positive,df_orig_predicted])

In [21]:

de_result1=df_outcome[df_outcome.iloc[:,-3].values==1]
de_result2=de_result1[de_result1.iloc[:,-2].values==-1]
de_result3=de_result2[de_result2.iloc[:,-1].values==1]
de_result3

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,y,class_test,pred


In [53]:
df_outcome.to_csv('D:/1研究生学习资料/实验数据/实验数据/重做/重做-去重/Re去重/关键基因/RNA39作为未证实/PU_bagging_MLP_final_predict_调参3.csv',index=False)